In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 

# 커널을 구성하다보면 에러는 아니지만, 빨간색 네모 박스 warning이 뜨는 경우가 많다. 
# 그런 보기 싫은 부분들을 제거해주겠다.
# import warnings
# warnings.filterwarnings('ignore')

# os 패키지를 통해 현재 디렉토리 위치를 변경하고, read_csv를 더 편리하게 할 수 있음
import os
os.getcwd() # 현재 디렉토리 파악
# os.chdir(r"______") # 불러오고 싶은 파일이 위치한 주소를 ___에 입력

# 다른 노트북 작성할 때도 이 셀만 떼서 사용 가능하다.
import platform                

# 웬만하면 해주는 것이 좋다
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin': #맥os 사용자의 경우에
    plt.style.use('seaborn-darkgrid') 
    rc('font', family = 'AppleGothic')
    
elif platform.system() == 'Windows':#윈도우 사용자의 경우에
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    plt.style.use('seaborn-v0_8-whitegrid') # https://python-graph-gallery.com/199-matplotlib-style-sheets/
    rc('font', family=font_name)
    
# tqdm
from tqdm.notebook import tqdm

In [3]:
path = 'c:/reposit/data/dacon/2023 전력사용량 예측 AI 경진대회/'

In [15]:
path = 'c:/reposit/data/dacon/2023 전력사용량 예측 AI 경진대회/'

import random
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings(action='ignore') 

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

# 데이터 로드
train_df = pd.read_csv(path + 'train.csv')
test_df = pd.read_csv(path + 'test.csv')
building_df = pd.read_csv(path + 'building_info.csv')

# building info 전처리
building_df['태양광용량(kW)'] = building_df['태양광용량(kW)'].str.replace('-', '0')
building_df['ESS저장용량(kWh)'] = building_df['ESS저장용량(kWh)'].str.replace('-', '0')
building_df['PCS용량(kW)'] = building_df['PCS용량(kW)'].str.replace('-', '0')

building_df['태양광용량(kW)'] = building_df['태양광용량(kW)'].astype(float)
building_df['ESS저장용량(kWh)'] = building_df['ESS저장용량(kWh)'].astype(float)
building_df['PCS용량(kW)'] = building_df['PCS용량(kW)'].astype(float)


# 조인
train_df = pd.merge(train_df, building_df, on='건물번호', how='left')
test_df = pd.merge(test_df, building_df, on='건물번호', how='left')



#결측값을 0으로 채웁니다
train_df = train_df.fillna(0)

#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
train_df['month'] = train_df['일시'].apply(lambda x : int(x[4:6]))
train_df['day'] = train_df['일시'].apply(lambda x : int(x[6:8]))
train_df['time'] = train_df['일시'].apply(lambda x : int(x[9:11]))

train_x = train_df.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)', '건물유형'])
train_y = train_df['전력소비량(kWh)']

model = RandomForestRegressor()
model.fit(train_x, train_y)

test_df['month'] = test_df['일시'].apply(lambda x : int(x[4:6]))
test_df['day'] = test_df['일시'].apply(lambda x : int(x[6:8]))
test_df['time'] = test_df['일시'].apply(lambda x : int(x[9:11]))

test_x = test_df.drop(columns=['num_date_time', '일시', '건물유형'])

preds = model.predict(test_x)

submission = pd.read_csv(path + 'sample_submission.csv')
submission['answer'] = preds

submission.to_csv(path + 'submission_4.csv', index=False)

In [5]:
train_df = pd.read_csv(path + 'train.csv')
test_df = pd.read_csv(path + 'test.csv')
building_df = pd.read_csv(path + 'building_info.csv')

In [7]:
building_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   건물번호          100 non-null    int64  
 1   건물유형          100 non-null    object 
 2   연면적(m2)       100 non-null    float64
 3   냉방면적(m2)      100 non-null    float64
 4   태양광용량(kW)     100 non-null    object 
 5   ESS저장용량(kWh)  100 non-null    object 
 6   PCS용량(kW)     100 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 5.6+ KB


In [13]:
building_df['태양광용량(kW)'] = building_df['태양광용량(kW)'].str.replace('-', '0')
building_df['ESS저장용량(kWh)'] = building_df['ESS저장용량(kWh)'].str.replace('-', '0')
building_df['PCS용량(kW)'] = building_df['PCS용량(kW)'].str.replace('-', '0')

building_df['태양광용량(kW)'] = building_df['태양광용량(kW)'].astype(float)
building_df['ESS저장용량(kWh)'] = building_df['ESS저장용량(kWh)'].astype(float)
building_df['PCS용량(kW)'] = building_df['PCS용량(kW)'].astype(float)

building_df

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.00,39570.00,0.0,0.0,0.0
1,2,건물기타,122233.47,99000.00,0.0,0.0,0.0
2,3,건물기타,171243.00,113950.00,40.0,0.0,0.0
3,4,건물기타,74312.98,34419.62,60.0,0.0,0.0
4,5,건물기타,205884.00,150000.00,0.0,2557.0,1000.0
...,...,...,...,...,...,...,...
95,96,호텔및리조트,93314.00,60500.00,0.0,0.0,0.0
96,97,호텔및리조트,55144.67,25880.00,0.0,0.0,0.0
97,98,호텔및리조트,53578.62,17373.75,0.0,0.0,0.0
98,99,호텔및리조트,53499.00,40636.00,0.0,0.0,0.0


In [5]:
# 데이터 로드
train_df = pd.read_csv(path + 'train.csv')
test_df = pd.read_csv(path + 'test.csv')
building_df = pd.read_csv(path + 'building_info.csv')

# building info 전처리
building_df['태양광용량(kW)'] = building_df['태양광용량(kW)'].str.replace('-', '0')
building_df['ESS저장용량(kWh)'] = building_df['ESS저장용량(kWh)'].str.replace('-', '0')
building_df['PCS용량(kW)'] = building_df['PCS용량(kW)'].str.replace('-', '0')

building_df['태양광용량(kW)'] = building_df['태양광용량(kW)'].astype(float)
building_df['ESS저장용량(kWh)'] = building_df['ESS저장용량(kWh)'].astype(float)
building_df['PCS용량(kW)'] = building_df['PCS용량(kW)'].astype(float)


# 조인
train_df = pd.merge(train_df, building_df, on='건물번호', how='left')
test_df = pd.merge(test_df, building_df, on='건물번호', how='left')

In [7]:
train_df.head(3)

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28,건물기타,110634.0,39570.0,0.0,0.0,0.0
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36,건물기타,110634.0,39570.0,0.0,0.0,0.0
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88,건물기타,110634.0,39570.0,0.0,0.0,0.0
